# import

In [235]:
import pandas as pd

# ex1

## (1) 주어진 데이터에 총 몇개의 도시와 몇 개의 나라가 있는지 출력

### sol1

In [236]:
df = pd.read_csv('data/world_cities.csv')

print(f'{len(df["City / Urban area"].unique())}/{len(df["Country"].unique())}') 

249/61


### sol2

In [237]:
df['City / Urban area'].value_counts().shape

(249,)

In [238]:
df['Country'].value_counts().shape

(61,)

## (2) 인구 밀도(명/sqKm) 가 10000 이 넘는 도시는 총 몇 개인지 확인

### sol1

In [239]:
sum((df["Population"]/df["Land area (in sqKm)"])>10000)

19

## (3) 인구 밀도가 가장 높은 도시

### sol1

In [240]:
max_value = max(df["Population"]/df["Land area (in sqKm)"])

In [241]:
df.loc[df["Population"]/df["Land area (in sqKm)"] == max_value, "City / Urban area"]

75    Mumbai
Name: City / Urban area, dtype: object

## (4) 도시가 4개인 나라 출력

### sol

In [242]:
df["Country"].value_counts()[df["Country"].value_counts() == 4]

Country
Italy    4
Name: count, dtype: int64

***

# ex2. 조건에 부합하는지를 구별하는 컬럼 추가

`1` “information technology” 과목은 심화과목이라 1학년은 수강 x

`2` “commerce” 과목은 기초과목이고 많은 학생들이 듣는 수업이라 4학년은 수강 x

`3` 수강생이 5명이 되지 않으면 강의는 폐강되어 수강 x

### sol1

In [243]:
df = pd.read_csv('data/enrolment_1.csv')

In [244]:
df["status"] = ["not allowed" if ((i == 1) & (j == "information technology") | (i == 4) & (j == "commerce")) else "allowed" for i, j in zip(df.year,df["course name"])]

In [245]:
v_count = df.loc[df.status == "allowed"]["course name"].value_counts()

In [246]:
f = v_count[v_count<5].index

In [247]:
df.loc[map(lambda x : x in f, df["course name"]), "status"] = "not allowed"

In [248]:
df.head()

,id,year,course name,status
0,2777729,1,information technology,not allowed
1,2777730,2,science,allowed
2,2777765,1,arts,allowed
3,2777766,2,arts,allowed
4,2777785,1,mba,allowed


### sol2

In [249]:
import pandas as pd

df = pd.read_csv('data/enrolment_1.csv')
df["status"] = "allowed"

# 조건 1
boolean1 = df["course name"] == "information technology"
boolean2 = df["year"] == 1
df.loc[boolean1 & boolean2, "status"] = "not allowed"

# 조건 2
boolean3= df["course name"] == "commerce"
boolean4= df["year"] == 4
df.loc[boolean3& boolean4, "status"] = "not allowed"

# 조건 3
allowed = df["status"] == "allowed"
course_counts = df.loc[allowed, "course name"].value_counts()
closed_courses = list(course_counts[course_counts < 5].index)
for course in closed_courses:
    df.loc[df["course name"] == course, "status"] = "not allowed"

# 테스트 코드
df

,id,year,course name,status
0,2777729,1,information technology,not allowed
1,2777730,2,science,allowed
2,2777765,1,arts,allowed
3,2777766,2,arts,allowed
4,2777785,1,mba,allowed
...,...,...,...,...
1995,2796805,3,computer application,allowed
1996,2796812,1,nursing,allowed
1997,2796813,2,nursing,allowed
1998,2796814,3,nursing,allowed


`-` 접근방식은 같은데, `sol2`가 확실히 코드 가독성이 좋다...

***

# ex3. 조건에 따라 문제 해결

1.  80명 이상의 학생이 수강하는 과목은 “Auditorium”에서 진행됩니다.

2. 40명 이상, 80명 미만의 학생이 수강하는 과목은 “Large room”에서 진행됩니다.

3. 15명 이상, 40명 미만의 학생이 수강하는 과목은 “Medium room”에서 진행됩니다.

4. 5명 이상, 15명 미만의 학생이 수강하는 과목은 “Small room”에서 진행됩니다.

5. 폐강 등의 이유로 status가 “not allowed”인 수강생은 room assignment 또한 “not assigned”가 되어야 합니다.

### sol1

In [250]:
import pandas as pd

df = pd.read_csv('data/enrolment_2.csv')

`1` 폐강되지 않은 강의만 표시

In [251]:
allowed = df.status == "allowed"

`2` 학생 수 count

In [252]:
s_count = df.loc[allowed]["course name"].value_counts()
s_count = pd.DataFrame(s_count).reset_index()

`3` 컬럼생성

In [253]:
s_count.loc[s_count["count"] >= 80 ,"room assignment"] = "Auditorium"
s_count.loc[(s_count["count"]>=40) & (s_count["count"]<80)  ,"room assignment"] = "Large room"
s_count.loc[(s_count["count"]>=15) & (s_count["count"]<40)  ,"room assignment"] = "Medium room"
s_count.loc[(s_count["count"]>=5) & (s_count["count"]<15)  ,"room assignment"] = "Small room"

`4` 2개 테이블 조인

In [254]:
df = pd.merge(df, s_count, on = "course name")

In [255]:
df.loc[df.status == "not allowed", "room assignment"] = "not assigned"

In [256]:
df

,id,year,course name,status,count,room assignment
0,2777729,1,information technology,not allowed,13,not assigned
1,2777730,2,science,allowed,123,Auditorium
2,2777765,1,arts,allowed,158,Auditorium
3,2777766,2,arts,allowed,158,Auditorium
4,2777785,1,mba,allowed,6,Small room
...,...,...,...,...,...,...
1453,2796805,3,computer application,allowed,16,Medium room
1454,2796812,1,nursing,allowed,36,Medium room
1455,2796813,2,nursing,allowed,36,Medium room
1456,2796814,3,nursing,allowed,36,Medium room


## sol2 

In [257]:
import pandas as pd

df = pd.read_csv('data/enrolment_2.csv')

# 과목별 인원 가져오기
allowed = df["status"] == "allowed"
course_counts = df.loc[allowed, "course name"].value_counts()

# 각 강의실 규모에 해당되는 과목 리스트 만들기
auditorium_list = list(course_counts[course_counts >= 80].index)
large_room_list = list(course_counts[(80 > course_counts) & (course_counts >= 40)].index)
medium_room_list = list(course_counts[(40 > course_counts) & (course_counts >= 15)].index)
small_room_list = list(course_counts[(15 > course_counts) & (course_counts > 4)].index)

# not allowed 과목에 대해 값 지정해주기
not_allowed = df["status"] == "not allowed"
df.loc[not_allowed, "room assignment"] = "not assigned"

# allowed 과목에 대해 값 지정해주기
for course in auditorium_list:
    df.loc[(df["course name"] == course) & allowed, "room assignment"] = "Auditorium"

for course in large_room_list:
    df.loc[(df["course name"] == course) & allowed, "room assignment"] = "Large room"
    
for course in medium_room_list:
    df.loc[(df["course name"] == course) & allowed, "room assignment"] = "Medium room"
    
for course in small_room_list:
    df.loc[(df["course name"] == course) & allowed, "room assignment"] = "Small room"
    
# 정답 출력
df

,id,year,course name,status,room assignment
0,2777729,1,information technology,not allowed,not assigned
1,2777730,2,science,allowed,Auditorium
2,2777765,1,arts,allowed,Auditorium
3,2777766,2,arts,allowed,Auditorium
4,2777785,1,mba,allowed,Small room
...,...,...,...,...,...
1995,2796805,3,computer application,allowed,Medium room
1996,2796812,1,nursing,allowed,Medium room
1997,2796813,2,nursing,allowed,Medium room
1998,2796814,3,nursing,allowed,Medium room


`-` 큼.. 이건 내 코드가 더 효율적인 것 같음(가독성도!)

***

# ex4. 조건에 따라 문제 해결

`1` 같은 크기의 강의실이 필요한 과목에 대해 알파벳 순서대로 방 번호를 배정하세요.

* 예를 들어 "Auditorium"이 필요한 과목으로 “arts”, “commerce”, “science” 세 과목이 있다면, “arts”는 “Auditorium-1”, “commerce”는 “Auditorium-2”, “science”는 “Auditorium-3” 순서로 방 배정이 되어야 합니다.

*  방 번호에 room 은 포함되지 않습니다. 아래 스크린샷을 참고하여 작성해주세요.

`2` status column이 “not allowed”인 수강생은 room assignment column을 그대로 “not assigned”로 남겨둡니다. "not allowed" 인 수강생의 room assignment 상태가 변경되지 않도록 유의해주세요.

`3` room assignment column의 이름을 room number로 바꿔주세요.

## sol1

In [258]:
import pandas as pd
import warnings

warnings.filterwarnings(action = "ignore")
df = pd.read_csv('data/enrolment_3.csv')

In [259]:
allowed = df.status == "allowed"

`-` 고유값만 출력

In [260]:
u_df = df.loc[allowed,  ["course name", "room assignment"]].\
                drop_duplicates().sort_values(["room assignment","course name"])

In [261]:
u_df.head()

,course name,room assignment
2,arts,Auditorium
60,commerce,Auditorium
1,science,Auditorium
202,education,Large room
24,english,Large room


`-` room_index

In [262]:
room_index = u_df["room assignment"].value_counts().index
room_values = u_df["room assignment"].value_counts().values

room_index, room_values

(Index(['Small room', 'Medium room', 'Auditorium', 'Large room'], dtype='object', name='room assignment'),
 array([53, 24,  3,  2], dtype=int64))

`-` 룸 넘버링

In [263]:
new_df = pd.DataFrame()
for k in range(4) :
    temp = u_df.loc[u_df["room assignment"] == room_index[k],:]
    temp["room number"] = [i + "-" + str(j) for i,j in zip(temp["room assignment"], range(1, room_values[k]+1))]
    new_df = pd.concat([new_df,temp], axis = 0)

In [264]:
new_df = new_df[["course name","room number"]]

In [265]:
df1 = pd.merge(df, new_df, on = "course name",how = "left")

In [266]:
df1.loc[df1["room assignment"] == "not assigned", "room number"] = "not assigned"

In [267]:
df1["room number"] = [i.replace(" room", "") for i in df1["room number"]]
df = df1.drop("room assignment", axis = 1)
df

,id,year,course name,status,room number
0,2777729,1,information technology,not allowed,not assigned
1,2777730,2,science,allowed,Auditorium-3
2,2777765,1,arts,allowed,Auditorium-1
3,2777766,2,arts,allowed,Auditorium-1
4,2777785,1,mba,allowed,Small-34
...,...,...,...,...,...
1995,2796805,3,computer application,allowed,Medium-7
1996,2796812,1,nursing,allowed,Medium-22
1997,2796813,2,nursing,allowed,Medium-22
1998,2796814,3,nursing,allowed,Medium-22


### 전체코드

In [269]:
import pandas as pd

df = pd.read_csv('data/enrolment_3.csv')

# 여기에 코드를 작성하세요
allowed = df.status == "allowed"
u_df = df.loc[allowed,  ["course name", "room assignment"]].\
                drop_duplicates().sort_values(["room assignment","course name"])
room_index = u_df["room assignment"].value_counts().index
room_values = u_df["room assignment"].value_counts().values

new_df = pd.DataFrame()
for k in range(4) :
    temp = u_df.loc[u_df["room assignment"] == room_index[k],:]
    temp["room number"] = [i + "-" + str(j) for i,j in zip(temp["room assignment"], range(1, room_values[k]+1))]
    new_df = pd.concat([new_df,temp], axis = 0)
new_df = new_df[["course name","room number"]]
df1 = pd.merge(df, new_df, on = "course name",how = "left")
df1.loc[df1["room assignment"] == "not assigned", "room number"] = "not assigned"
df1["room number"] = [i.replace(" room", "") for i in df1["room number"]]
df = df1.drop("room assignment", axis = 1)
df

,id,year,course name,status,room number
0,2777729,1,information technology,not allowed,not assigned
1,2777730,2,science,allowed,Auditorium-3
2,2777765,1,arts,allowed,Auditorium-1
3,2777766,2,arts,allowed,Auditorium-1
4,2777785,1,mba,allowed,Small-34
...,...,...,...,...,...
1995,2796805,3,computer application,allowed,Medium-7
1996,2796812,1,nursing,allowed,Medium-22
1997,2796813,2,nursing,allowed,Medium-22
1998,2796814,3,nursing,allowed,Medium-22


## sol2

In [268]:
import pandas as pd

df = pd.read_csv('data/enrolment_3.csv')

# 과목별 인원 가져오기
allowed = df["status"] == "allowed"
course_counts = df.loc[allowed, "course name"].value_counts()

# 각 강의실 규모에 해당되는 과목 리스트 만들기
auditorium_list = list(course_counts[course_counts >= 80].index)
large_room_list = list(course_counts[(80 > course_counts) & (course_counts >= 40)].index)
medium_room_list = list(course_counts[(40 > course_counts) & (course_counts >= 15)].index)
small_room_list = list(course_counts[(15 > course_counts) & (course_counts > 4)].index)

# 강의실 이름 붙이기
for i in range(len(auditorium_list)):
    df.loc[(df["course name"] == sorted(auditorium_list)[i]) & allowed, "room assignment"] = "Auditorium-" + str(i + 1)

for i in range(len(large_room_list)):
    df.loc[(df["course name"] == sorted(large_room_list)[i]) & allowed, "room assignment"] = "Large-" + str(i + 1)
    
for i in range(len(medium_room_list)):
    df.loc[(df["course name"] == sorted(medium_room_list)[i]) & allowed, "room assignment"] = "Medium-" + str(i + 1)
    
for i in range(len(small_room_list)):
    df.loc[(df["course name"] == sorted(small_room_list)[i]) & allowed, "room assignment"] = "Small-" + str(i + 1)

# column 이름 바꾸기
df.rename(columns={"room assignment": "room number"}, inplace = True)
    
# 테스트 코드
df

,id,year,course name,status,room number
0,2777729,1,information technology,not allowed,not assigned
1,2777730,2,science,allowed,Auditorium-3
2,2777765,1,arts,allowed,Auditorium-1
3,2777766,2,arts,allowed,Auditorium-1
4,2777785,1,mba,allowed,Small-34
...,...,...,...,...,...
1995,2796805,3,computer application,allowed,Medium-7
1996,2796812,1,nursing,allowed,Medium-22
1997,2796813,2,nursing,allowed,Medium-22
1998,2796814,3,nursing,allowed,Medium-22
